# Importing Libraries

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(1337)  # for reproducibility
import csv


from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.regularizers import l2, activity_l2
from keras.layers.advanced_activations import LeakyReLU



Using Theano backend.


In [2]:
os.chdir("/Users/Armin/Documents/Data_Science/Kaggle/Digit_Recognizer")
cwd=os.getcwd()
print "Working Dir Is:\n",cwd
print os.listdir(cwd+"/Data/")


Working Dir Is:
/Users/Armin/Documents/Data_Science/Kaggle/Digit_Recognizer
['.DS_Store', 'prediction_results.csv', 'test.csv', 'train.csv']


In [3]:
train_data = csv.reader(open(cwd+"/Data/train.csv","rb"),delimiter=',')
x = list(train_data)
train_XY= np.array(x[1:]).astype('float')
test_data = csv.reader(open(cwd+"/Data/test.csv","rb"),delimiter=',')
x = list(test_data)
test_XY= np.array(x[1:]).astype('float')

In [4]:
batch_size = 128
nb_classes = 10
nb_epoch = 10

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size
nb_conv = 3

In [5]:
train_percentage = 0.7
train_size = int(np.round(train_XY.shape[0]*train_percentage))

In [6]:
X_train = train_XY[0:train_size,1:]
print ("X_train size is:",X_train.shape)
y_train = train_XY[0:train_size,0]
print ("y_train size is:",y_train.shape)
X_test = train_XY[train_size:,1:   ]
print ("X_test size is:",X_test.shape)
y_test = train_XY[train_size:,0]
print ("y_test size is:",y_test.shape)

X_submission = test_XY
print ("X_submission size is:",X_submission.shape)

('X_train size is:', (29400, 784))
('y_train size is:', (29400,))
('X_test size is:', (12600, 784))
('y_test size is:', (12600,))
('X_submission size is:', (28000, 784))


In [7]:
X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
X_submission = X_submission.reshape(X_submission.shape[0], 1, img_rows, img_cols)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_submission = X_submission.astype('float32')
X_train /= 255
X_test /= 255
X_submission /= 255

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

('X_train shape:', (29400, 1, 28, 28))
(29400, 'train samples')
(12600, 'test samples')


In [8]:
y_train = np.asarray(y_train).astype('int8')
y_test = np.asarray(y_test).astype('int8')

In [9]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [15]:
model = Sequential()

model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                        border_mode='valid',
                        input_shape=(1, img_rows, img_cols)))
model.add(Activation(LeakyReLU(alpha=0.3)))
model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
model.add(Activation(LeakyReLU(alpha=0.3)))
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation(LeakyReLU(alpha=0.3)))
model.add(Dense(64))
model.add(Activation(LeakyReLU(alpha=0.3)))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])


In [ ]:
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 29400 samples, validate on 12600 samples
Epoch 1/10
29400/29400 [==============================] - 57s - loss: 0.3093 - acc: 0.9042 - val_loss: 0.1150 - val_acc: 0.9608
Epoch 2/10
29400/29400 [==============================] - 58s - loss: 0.1071 - acc: 0.9679 - val_loss: 0.0672 - val_acc: 0.9789
Epoch 3/10
29400/29400 [==============================] - 57s - loss: 0.0775 - acc: 0.9759 - val_loss: 0.0825 - val_acc: 0.9740
Epoch 4/10
29400/29400 [==============================] - 58s - loss: 0.0633 - acc: 0.9811 - val_loss: 0.0645 - val_acc: 0.9806
Epoch 5/10
29400/29400 [==============================] - 58s - loss: 0.0516 - acc: 0.9845 - val_loss: 0.0613 - val_acc: 0.9833
Epoch 6/10
29400/29400 [==============================] - 58s - loss: 0.0455 - acc: 0.9855 - val_loss: 0.0546 - val_acc: 0.9844
Epoch 7/10
29400/29400 [==============================] - 58s - loss: 0.0419 - acc: 0.9868 - val_loss: 0.0534 - val_acc: 0.9851
Epoch 8/10
29400/29400 [==============================

In [ ]:
prediction_results = model.predict_classes(X_submission, batch_size=32, verbose=1)

In [ ]:
prediction_results = prediction_results.astype('uint8')
print prediction_results

In [ ]:
np.savetxt("prediction_results.csv", prediction_results, delimiter=',')
np.savetxt('mnist-vggnet.csv', np.c_[range(1,len(yPred)+1),yPred], delimiter=',', header = 'ImageId,Label', comments = '', fmt='%d')